

---

# **Chapter 6: Adapters and Wrappers**

## **Opening Context**
Real-world software is never developed in isolation. You integrate third-party APIs, consume legacy libraries, and refactor existing code—all while maintaining backward compatibility. The resulting architectural challenge is **structural incompatibility**: interfaces that don't match, subsystems that expose overwhelming complexity, and objects that require controlled access. Structural patterns resolve these tensions without modifying existing code (OCP compliance). This chapter examines Adapter for interface harmonization, Facade for complexity reduction, and Proxy for access control—the three foundational wrappers that make integration possible.

---

## **6.1 Adapter Pattern: Bridging Incompatible Interfaces**

### **Intent**
*Convert the interface of a class into another interface that clients expect. Adapter lets classes work together that couldn't otherwise because of incompatible interfaces.*

### **The Problem**
You have a `MediaPlayer` application that expects audio files in a specific format:

```typescript
// Existing interface (client expectation)
interface MediaPlayer {
    play(filename: string): void;
}
```

A third-party library provides advanced audio decoding, but with a completely different interface:

```typescript
// Third-party library (incompatible interface)
class AdvancedAudioPlayer {
    playMp4(file: string): void {
        console.log(`Playing MP4: ${file}`);
    }
    
    playVlc(file: string): void {
        console.log(`Playing VLC: ${file}`);
    }
}
```

You cannot modify `AdvancedAudioPlayer` (closed source), but you must integrate it into your `MediaPlayer` architecture. Direct usage violates DIP and forces client code to depend on concrete implementations.

### **The Solution: Object Adapter (Preferred)**

The Adapter pattern wraps the adaptee (third-party class) and exposes the target interface (client expectation) through composition.

```typescript
/**
 * TARGET INTERFACE
 * What the client expects and uses.
 */
interface MediaPlayer {
    play(audioType: string, filename: string): void;
}

/**
 * ADAPTEE
 * The existing interface that needs adapting (third-party library).
 */
class AdvancedAudioPlayer {
    playMp4(filename: string): void {
        console.log(`Decoding MP4 codec: ${filename}`);
    }
    
    playVlc(filename: string): void {
        console.log(`Decoding VLC codec: ${filename}`);
    }
}

/**
 * ADAPTER
 * Implements Target interface, delegates to Adaptee.
 * Uses COMPOSITION (has-a relationship) - preferred over inheritance.
 */
class MediaAdapter implements MediaPlayer {
    private advancedPlayer: AdvancedAudioPlayer;
    
    constructor() {
        this.advancedPlayer = new AdvancedAudioPlayer();
    }
    
    /**
     * Adapts the Target interface (play) to Adaptee interface (playMp4/playVlc).
     * Handles translation of parameters and method names.
     */
    play(audioType: string, filename: string): void {
        if (audioType === "mp4") {
            this.advancedPlayer.playMp4(filename);
        } else if (audioType === "vlc") {
            this.advancedPlayer.playVlc(filename);
        } else {
            throw new Error(`Format ${audioType} not supported by adapter`);
        }
    }
}

/**
 * CLIENT
 * Works with Target interface (MediaPlayer), unaware of Adapter or Adaptee.
 */
class AudioPlayer implements MediaPlayer {
    play(audioType: string, filename: string): void {
        // Simple built-in support
        if (audioType === "mp3") {
            console.log(`Playing MP3 natively: ${filename}`);
        } 
        // Complex formats delegated through adapter
        else if (audioType === "mp4" || audioType === "vlc") {
            const adapter = new MediaAdapter();
            adapter.play(audioType, filename);
        } else {
            console.log(`Invalid media. ${audioType} format not supported`);
        }
    }
}

// Usage
const player = new AudioPlayer();
player.play("mp3", "song.mp3");      // Native
player.play("mp4", "movie.mp4");     // Adapted
player.play("vlc", "video.vlc");     // Adapted
```

**Code Explanation**:
- **Target** (`MediaPlayer`): The interface the client (`AudioPlayer`) understands.
- **Adaptee** (`AdvancedAudioPlayer`): The incompatible class with useful behavior.
- **Adapter** (`MediaAdapter`): Bridges the gap, translating `play(type, file)` into `playMp4(file)` or `playVlc(file)`.
- **Composition**: The adapter *has-a* `AdvancedAudioPlayer` (preferred) rather than *is-a* `AdvancedAudioPlayer` (class adapter).

### **Class Adapter Alternative (Avoid)**

Some languages (C++, Python) support multiple inheritance, enabling Class Adapter:

```typescript
// ❌ CLASS ADAPTER (Inheritance-based, NOT recommended)
class ClassAdapter extends AdvancedAudioPlayer implements MediaPlayer {
    play(audioType: string, filename: string): void {
        if (audioType === "mp4") this.playMp4(filename);
        // ...
    }
}
```

**Why Avoid Class Adapter**:
- Violates composition over inheritance principle
- Couples adapter to specific adaptee class (cannot adapt subclasses)
- Inflexible if adaptee changes implementation

### **Real-World Application: API Modernization**

Modernizing a legacy REST API to GraphQL without modifying the legacy system:

```typescript
// Legacy API Response (incompatible structure)
interface LegacyUserResponse {
    user_id: number;
    first_name: string;
    last_name: string;
    account_status: 0 | 1; // 0=inactive, 1=active
}

// Modern API expectation (Target interface)
interface ModernUser {
    id: string;
    fullName: string;
    isActive: boolean;
}

// Adapter transforming legacy to modern
class UserApiAdapter {
    constructor(private legacyApi: LegacyApiClient) {}
    
    async getUser(id: string): Promise<ModernUser> {
        const legacy = await this.legacyApi.fetchUser(parseInt(id));
        
        // Adaptation logic (data transformation)
        return {
            id: legacy.user_id.toString(),
            fullName: `${legacy.first_name} ${legacy.last_name}`,
            isActive: legacy.account_status === 1
        };
    }
}
```

### **Connection to SOLID**
- **OCP**: Extend system functionality (support new formats) without modifying existing `AudioPlayer` code.
- **DIP**: Client (`AudioPlayer`) depends on abstraction (`MediaPlayer`), not concrete `AdvancedAudioPlayer`.

---

## **6.2 Facade Pattern: Simplifying Complex Subsystems**

### **Intent**
*Provide a unified interface to a set of interfaces in a subsystem. Facade defines a higher-level interface that makes the subsystem easier to use.*

### **The Problem**
Modern frameworks and libraries expose fine-grained, powerful APIs that require extensive boilerplate for common operations. Consider sending an email:

```typescript
// ❌ WITHOUT FACADE: Client must coordinate multiple subsystems
class EmailClient {
    sendEmail(to: string, subject: string, body: string): void {
        // Must know about SMTP, authentication, MIME encoding, etc.
        const session = new SMTPSession();
        session.setHost("smtp.gmail.com");
        session.setPort(587);
        session.setTLS(true);
        
        const auth = new Authenticator();
        auth.setCredentials("user", "pass");
        session.setAuthenticator(auth);
        
        const message = new MimeMessage();
        message.setHeader("To", to);
        message.setHeader("Subject", subject);
        message.setContent(new MimeBody(body));
        
        const transport = new Transport(session);
        transport.connect();
        transport.send(message);
        transport.close();
    }
}
```

The client is coupled to the internal complexity of the email subsystem, violating the **Law of Demeter** (talk only to your immediate friends).

### **The Solution: Unified Interface**

The Facade provides a single entry point that encapsulates the subsystem's workflow.

```typescript
/**
 * FACADE
 * Simplified interface hiding complex subsystem interactions.
 */
class EmailFacade {
    constructor(
        private smtpConfig: SMTPConfig,
        private credentials: Credentials
    ) {}
    
    /**
     * Unified method hiding coordination complexity.
     * Client supplies only business-relevant data.
     */
    sendEmail(to: string, subject: string, body: string): void {
        // Internal coordination of subsystems
        const session = this.createSession();
        const message = this.createMessage(to, subject, body);
        
        const transport = new Transport(session);
        try {
            transport.connect();
            transport.send(message);
        } finally {
            transport.close();
        }
    }
    
    private createSession(): SMTPSession {
        const session = new SMTPSession();
        session.setHost(this.smtpConfig.host);
        session.setPort(this.smtpConfig.port);
        
        const auth = new Authenticator();
        auth.setCredentials(this.credentials.user, this.credentials.pass);
        session.setAuthenticator(auth);
        
        return session;
    }
    
    private createMessage(to: string, subject: string, body: string): MimeMessage {
        const message = new MimeMessage();
        message.setHeader("To", to);
        message.setHeader("Subject", subject);
        message.setContent(new MimeBody(body));
        return message;
    }
}

// Subsystem classes (complex, numerous)
class SMTPSession { setHost(h: string): void {} setPort(p: number): void {} setAuthenticator(a: Authenticator): void {} }
class Authenticator { setCredentials(u: string, p: string): void {} }
class MimeMessage { setHeader(k: string, v: string): void {} setContent(b: MimeBody): void {} }
class MimeBody { constructor(private content: string) {} }
class Transport {
    constructor(private session: SMTPSession) {}
    connect(): void { console.log("Connecting..."); }
    send(msg: MimeMessage): void { console.log("Sending..."); }
    close(): void { console.log("Closing..."); }
}

interface SMTPConfig { host: string; port: number; }
interface Credentials { user: string; pass: string; }

// Client usage (dramatically simplified)
class NotificationService {
    private emailFacade: EmailFacade;
    
    constructor() {
        this.emailFacade = new EmailFacade(
            { host: "smtp.gmail.com", port: 587 },
            { user: "app@example.com", pass: "secret" }
        );
    }
    
    notifyUser(email: string, message: string): void {
        // Single, semantic method call
        this.emailFacade.sendEmail(email, "Notification", message);
    }
}
```

**Code Explanation**:
- **Facade** (`EmailFacade`): Knows which subsystem classes are responsible for which actions and delegates client requests appropriately.
- **Subsystem Classes**: Maintain their complexity and independence; the facade does not encapsulate them (they remain accessible for advanced use).
- **Law of Demeter Compliance**: Client depends only on `EmailFacade`, not on `SMTPSession`, `MimeMessage`, etc.

### **Facade vs. Adapter**
| Facade | Adapter |
|--------|---------|
| Simplifies a complex **subsystem** | Converts one **interface** to another |
| Usually a new interface | Usually implements existing target interface |
| Multiple classes behind it | Usually single class behind it |
| Goal: Usability | Goal: Compatibility |

### **Warning: The "Leaky Facade" Anti-Pattern**
A Facade should not expose subsystem details through its interface:

```typescript
// ❌ LEAKY FACADE (Bad)
class BadFacade {
    getSMTPSession(): SMTPSession { return this.session; } // Exposes internals!
}

// ✅ PROPER FACADE (Good)
class GoodFacade {
    sendEmail(...): void { /* uses session internally only */ }
}
```

If clients need to access subsystems, provide optional access explicitly:
```typescript
class EmailFacade {
    // Primary simplified interface
    sendSimpleEmail(to: string, body: string): void { /* ... */ }
    
    // Optional: Expose advanced configuration for power users
    createAdvancedMessage(): MimeMessage { return new MimeMessage(); }
    sendRawMessage(msg: MimeMessage): void { /* ... */ }
}
```

---

## **6.3 Proxy Pattern: Controlling Access**

### **Intent**
*Provide a surrogate or placeholder for another object to control access to it.*

### **The Variants**
The Proxy pattern has multiple specialized forms based on the type of control needed:

1.  **Virtual Proxy**: Lazy initialization (defer expensive object creation).
2.  **Protection Proxy**: Access control (permissions, authentication).
3.  **Remote Proxy**: Local representative for remote object (RPC).
4.  **Smart Reference**: Additional actions on object access (reference counting, logging).

### **Implementation: Virtual Proxy (Lazy Loading)**

Expensive objects (high-res images, large database connections) should be created only when actually needed.

```typescript
/**
 * SUBJECT INTERFACE
 * Both RealSubject and Proxy implement this.
 */
interface Image {
    display(): void;
    getDimensions(): { width: number; height: number };
}

/**
 * REAL SUBJECT
 * The expensive object we want to defer creating.
 */
class HighResolutionImage implements Image {
    private filename: string;
    private data: Buffer; // Expensive resource
    
    constructor(filename: string) {
        this.filename = filename;
        this.loadFromDisk(); // EXPENSIVE OPERATION
    }
    
    private loadFromDisk(): void {
        console.log(`Loading ${this.filename} from disk (10MB)...`);
        // Simulate expensive I/O
        this.data = Buffer.alloc(10 * 1024 * 1024); 
    }
    
    display(): void {
        console.log(`Displaying ${this.filename}`);
    }
    
    getDimensions(): { width: number; height: number } {
        return { width: 3840, height: 2160 };
    }
}

/**
 * PROXY
 * Controls access to RealSubject, implements lazy initialization.
 */
class ImageProxy implements Image {
    private filename: string;
    private realImage: HighResolutionImage | null = null;
    private cachedDimensions: { width: number; height: number } | null = null;
    
    constructor(filename: string) {
        // Cheap operation - just store filename
        this.filename = filename;
        // Could load metadata only (cheap) for getDimensions
        this.cachedDimensions = this.loadMetadata(filename);
    }
    
    /**
     * Virtual Proxy pattern: Defer expensive creation until display() called.
     */
    display(): void {
        if (!this.realImage) {
            console.log("Proxy: Creating real image on first display...");
            this.realImage = new HighResolutionImage(this.filename);
        }
        this.realImage.display();
    }
    
    getDimensions(): { width: number; height: number } {
        // Return cached metadata without loading full image
        if (this.cachedDimensions) {
            return this.cachedDimensions;
        }
        return this.realImage?.getDimensions() || { width: 0, height: 0 };
    }
    
    private loadMetadata(filename: string): { width: number; height: number } {
        // Simulate cheap metadata read (EXIF data, etc.)
        console.log(`Proxy: Loading metadata for ${filename} (cheap)`);
        return { width: 3840, height: 2160 };
    }
}

// Usage
class ImageGallery {
    private images: Image[] = [];
    
    addImage(filename: string): void {
        // Fast - no actual loading happens
        this.images.push(new ImageProxy(filename));
    }
    
    showThumbnails(): void {
        // Only uses getDimensions() - no expensive loading
        this.images.forEach(img => {
            const dims = img.getDimensions();
            console.log(`Thumbnail: ${dims.width}x${dims.height}`);
        });
    }
    
    viewFullImage(index: number): void {
        // NOW we pay the cost, but only for this specific image
        this.images[index].display();
    }
}

// Client
const gallery = new ImageGallery();
gallery.addImage("vacation1.jpg"); // Instant, no load
gallery.addImage("vacation2.jpg"); // Instant, no load
gallery.addImage("vacation3.jpg"); // Instant, no load

gallery.showThumbnails(); // Fast - only metadata
gallery.viewFullImage(0); // Expensive load happens HERE, only for image 0
```

**Code Explanation**:
- **Subject** (`Image`): Interface that both real object and proxy implement.
- **RealSubject** (`HighResolutionImage`): Expensive to create (simulated 10MB allocation).
- **Proxy** (`ImageProxy`): Maintains reference to RealSubject, creates it only when `display()` is called. Can provide lightweight operations (metadata) without full instantiation.

### **Implementation: Protection Proxy (Access Control)**

```typescript
interface Document {
    read(): string;
    write(content: string): void;
}

class ConfidentialDocument implements Document {
    constructor(private content: string) {}
    
    read(): string { return this.content; }
    write(content: string): void { this.content = content; }
}

class SecureDocumentProxy implements Document {
    private realDocument: ConfidentialDocument;
    private requiredRole: string;
    
    constructor(content: string, private userRole: string) {
        this.realDocument = new ConfidentialDocument(content);
        this.requiredRole = "ADMIN";
    }
    
    read(): string {
        if (this.checkAccess()) {
            console.log("Proxy: Access granted for reading");
            return this.realDocument.read();
        }
        throw new Error("Access Denied: Insufficient privileges to read");
    }
    
    write(content: string): void {
        if (this.checkAccess()) {
            console.log("Proxy: Access granted for writing");
            this.realDocument.write(content);
        } else {
            throw new Error("Access Denied: Insufficient privileges to write");
        }
    }
    
    private checkAccess(): boolean {
        return this.userRole === this.requiredRole;
    }
}

// Usage
const userDoc = new SecureDocumentProxy("Secret Data", "USER");
try {
    userDoc.read(); // Throws "Access Denied"
} catch (e) {
    console.log(e);
}

const adminDoc = new SecureDocumentProxy("Secret Data", "ADMIN");
console.log(adminDoc.read()); // Success
```

### **Proxy vs. Decorator**
Both wrap objects and implement the same interface, but:
- **Proxy**: Controls access (creation, permission, location). The interface usually remains semantically identical.
- **Decorator**: Adds responsibilities/behavior. The interface may be enhanced with new capabilities (Chapter 7).

---

## **Chapter Summary**
In this chapter, we explored three structural wrappers:

1.  **Adapter**: Converts incompatible interfaces to work together. Uses composition to wrap adaptee. Essential for legacy integration and third-party APIs. **Status: Essential for integration architecture.**

2.  **Facade**: Provides simplified interface to complex subsystems. Reduces coupling between clients and subsystem classes, following the Law of Demeter. Must not leak subsystem details. **Status: Critical for API design and framework usability.**

3.  **Proxy**: Controls access to objects through surrogates. Virtual Proxy defers expensive creation; Protection Proxy enforces security; Remote Proxy handles distribution. Distinguishable from Decorator by intent (access control vs. behavior addition). **Status: Fundamental for performance optimization and security.**

**Key Distinctions**:
- **Adapter** changes interface to match expectation.
- **Facade** simplifies complex workflows.
- **Proxy** controls access to the real object.

---

## **Next Chapter Preview**
**Chapter 7: Dynamic Extensions and Aggregation (Decorator, Composite, Bridge, Flyweight)**

Continuing with Structural patterns, Chapter 7 explores advanced composition techniques. We examine the **Decorator** pattern for adding responsibilities dynamically without subclassing, the **Composite** pattern for treating individual objects and compositions uniformly (tree structures), the **Bridge** pattern for decoupling abstraction from implementation (essential for platform-independent code), and the **Flyweight** pattern for efficient sharing of large numbers of fine-grained objects. These patterns address the challenge of building flexible, scalable object structures that evolve at runtime.

---

